In [442]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

In [443]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [444]:
tf.__version__

'1.13.1'

In [445]:
tf.keras.__version__

'2.2.4-tf'

## IMPORT TWEET DATA


In [446]:
#Import
dem_train_text = pd.read_csv('../data_master/nlp_feed_data/dem_tweets.csv')                        

rep_test_text = pd.read_csv('../data_master/nlp_feed_data/rep_tweets.csv')

#Drop and show
dem_train_text = dem_train_text.drop(['Unnamed: 0'], axis=1)
print('DEMOCRAT TRAIN TWEETS')
print(dem_train_text.head())
print(type(dem_train_text)) 
      
rep_test_text = rep_test_text.drop(['Unnamed: 0'], axis=1)
print('REPUBLICAN TEST TWEETS')
print(rep_test_text.head())
print(type(rep_test_text))


DEMOCRAT TRAIN TWEETS
                                               tweet
0  We're in a battle for the soul of this nation....
1  Today marks 50 years since the Stonewall upris...
2  I’ve fought my heart out to ensure that civil ...
3  At last night’s first Democratic debate, I lai...
4  I just got off the debate stage, where I laid ...
<class 'pandas.core.frame.DataFrame'>
REPUBLICAN TEST TWEETS
                                               tweet
0  NO American should worry about the cost of lif...
1  ALL American families deserve to live in safe ...
2  A targeted attack on our democracy is no laugh...
3  House Democrats are FOCUSED on protecting our ...
4  Mental health care is health care. We must pro...
<class 'pandas.core.frame.DataFrame'>


## CONVERT DATAFRAMES TO LIST FOR PREP FEED TO NLP

In [447]:
rep_test_text = rep_test_text.values.tolist()
for i in range(len(rep_test_text)):
    rep_test_text[i] = str(rep_test_text[i])
dem_train_text = dem_train_text.values.tolist()
for i in range(len(dem_train_text)):
    dem_train_text[i] = str(dem_train_text[i])
print('REPUBLICAN TEST LIST LENGTH')
print(len([rep_test_text]))
print(type(rep_test_text))
print('DEMOCRAT TRAIN LIST LENGTH')
print(len([dem_train_text]))
print(type(dem_train_text))

REPUBLICAN TEST LIST LENGTH
1
<class 'list'>
DEMOCRAT TRAIN LIST LENGTH
1
<class 'list'>


## IMPORT CLASSIFIER DATA

In [448]:
#Import
#dem_train_y = pd.read_csv('../data_master/nlp_feed_data/dem_classifier.csv')                        

#rep_train_x = pd.read_csv('../data_master/nlp_feed_data/rep_classifier.csv')

#Drop and show
#dem_train_y = dem_train_y.drop(['Unnamed: 0'], axis=1)
#print('DEMOCRAT CLASSIFIER')
#print(dem_train_y.head())
#print(type(dem_train_y))
      
#rep_train_x = rep_train_x.drop(['Unnamed: 0'], axis=1)
#print('REPUBLICAN CLASSIFIER')
#print(rep_train_x.head())
#print(type(rep_train_x))


## CONVERT DATAFRAMES TO LIST FOR PREP FEED TO NLP

In [449]:
#rep_train_x = rep_train_x.values.tolist()
#for i in range(len(rep_train_x)):
#    rep_train_x[i] = str(rep_train_x[i])
#dem_train_y = dem_train_y.values.tolist()

#print('REPUBLICAN LIST LENGTH')
#print(len([rep_train_x]))
#print(type(rep_train_x))
#print('DEMOCRAT LIST LENGTH')
#print(len([dem_train_y]))
#print(type(dem_train_y))


In [455]:
dem_train_y = [i for i in np.arange(1,9055,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(dem_train_y[7])
for n, i in enumerate(dem_train_y):
              dem_train_y[n] = 0
print('Interger Return off of DF refactored (0) build')
print(dem_train_y[7])
dem_train_y

rep_test_x = [i for i in np.arange(1,14449,1)]
print('Interger Return off of DF initial build (7th posistion)')
print(rep_test_x[7])
for n, i in enumerate(rep_test_x):
              rep_test_x[n] = 1
print('Interger Return off of DF refactored (1) build')
print(rep_test_x[7])

Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (0) build
0
Interger Return off of DF initial build (7th posistion)
8
Interger Return off of DF refactored (1) build
1


In [456]:
#NEED TO REPLACE SECOND rep_train_text WITH TEST DATA (TWEETS)
data_text = dem_train_text + rep_test_text

In [457]:
#rep_train_text[1]
#print(type(rep_train_text[1]))
#for i in range(len(rep_train_text)):
#    rep_train_text[i] = str(rep_train_text[i])
    
#rep_train_text

In [458]:
num_words = 10000

In [459]:
tokenizer = Tokenizer(num_words=num_words)

In [460]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 839 ms, sys: 8.32 ms, total: 847 ms
Wall time: 851 ms


In [461]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [462]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'co': 3,
 't': 4,
 'https': 5,
 'of': 6,
 'a': 7,
 'and': 8,
 'in': 9,
 'for': 10,
 'is': 11,
 'amp': 12,
 'n': 13,
 "'": 14,
 'on': 15,
 'that': 16,
 'we': 17,
 'our': 18,
 'with': 19,
 'are': 20,
 'this': 21,
 'will': 22,
 'be': 23,
 'it': 24,
 'have': 25,
 'you': 26,
 'i': 27,
 'they': 28,
 'not': 29,
 'from': 30,
 'by': 31,
 'at': 32,
 'their': 33,
 'has': 34,
 'people': 35,
 'as': 36,
 'all': 37,
 'who': 38,
 'but': 39,
 'he': 40,
 'more': 41,
 'about': 42,
 'his': 43,
 'was': 44,
 'an': 45,
 'us': 46,
 'your': 47,
 'or': 48,
 'my': 49,
 's': 50,
 "'the": 51,
 'up': 52,
 'now': 53,
 '”': 54,
 'no': 55,
 'nhttps': 56,
 'do': 57,
 'u': 58,
 'out': 59,
 'them': 60,
 'must': 61,
 'today': 62,
 'country': 63,
 'president': 64,
 'can': 65,
 '–': 66,
 'time': 67,
 'just': 68,
 'american': 69,
 'should': 70,
 'one': 71,
 'americans': 72,
 'what': 73,
 'great': 74,
 'so': 75,
 'make': 76,
 'venezuela': 77,
 'if': 78,
 'get': 79,
 'trump': 80,
 'need': 81,
 'tax': 82,


In [463]:
dem_text_train_tokens = tokenizer.texts_to_sequences(dem_train_text)

In [464]:
np.array(dem_text_train_tokens[1])

array([ 244, 4000,  829,  105,  300,    1, 6221,  240,  110, 1117, 2517,
        681,    7,  165,   15, 4165, 1624,    8, 2191,   18,  166, 1749,
         17,   61,  222,   18,  122,   10,  350, 1429, 1240,  355,  288,
         11, 1262,   19,    1,  743,    8,  861,   28,  352,    5,    4,
          3])

In [465]:
#NEED TO REPLACE WITH TEST DATA (TWEETS)
rep_test_tokens = tokenizer.texts_to_sequences(rep_test_text)

## PADDING AND TRUNCATING DATA

In [466]:
num_tokens = [len(tokens) for tokens in dem_text_train_tokens + rep_test_tokens]
num_tokens = np.array(num_tokens)


In [467]:
np.mean(num_tokens)

30.565441239043487

In [468]:
np.max(num_tokens)

62

In [469]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

56

In [470]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9950217002808271

## NVM ACTUAL PADDING HERE

In [472]:
pad = 'pre'

In [473]:
x_train_pad = pad_sequences(dem_text_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [474]:
x_test_pad = pad_sequences(rep_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [475]:
x_train_pad.shape

(9054, 56)

In [476]:
x_test_pad.shape

(14448, 56)

In [477]:
np.array(rep_text_train_tokens[1])

array([1810,   81,  134,  707,    2,  187,    7,  449,  507,   10, 2119,
         91,   27,  367, 6395, 5639, 1082,    2,  799,   20, 3414,  635,
         13, 1811,    2,  209,   35,  268,   19,  947,   12,   87,  190,
          2,   60,   38,  185,   10,   47,    4,    3])

In [412]:
rep_text_train_tokens[1]

[1810,
 81,
 134,
 707,
 2,
 187,
 7,
 449,
 507,
 10,
 2119,
 91,
 27,
 367,
 6395,
 5639,
 1082,
 2,
 799,
 20,
 3414,
 635,
 13,
 1811,
 2,
 209,
 35,
 268,
 19,
 947,
 12,
 87,
 190,
 2,
 60,
 38,
 185,
 10,
 47,
 4,
 3]

## TOKENIZER INVERSE MAP

In [478]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [479]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [480]:
tokens_to_string(dem_text_train_tokens[1])

"'today marks 50 years since the uprising where those brave individuals took a stand on christopher st and changed our world forever we must continue our fight for full lgbtq equality until everyone is treated with the dignity and respect they deserve https t co"

## CREATE THE RECURRENT NEURAL NETWORK

In [481]:
model = Sequential()

In [482]:
embedding_size = 8

In [483]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [484]:
model.add(GRU(units=16, return_sequences=True))

In [485]:
model.add(GRU(units=8, return_sequences=True))

In [486]:
model.add(GRU(units=4))

In [487]:
model.add(Dense(1, activation='sigmoid'))

In [488]:
optimizer = Adam(lr=1e-3)

In [489]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [490]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 56, 8)             80000     
_________________________________________________________________
gru_15 (GRU)                 (None, 56, 16)            1200      
_________________________________________________________________
gru_16 (GRU)                 (None, 56, 8)             600       
_________________________________________________________________
gru_17 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


## TRAIN THE RECURRENT NERUAL NETWORK

In [491]:
%%time
model.fit(x_train_pad, dem_train_y,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 8601 samples, validate on 453 samples
Epoch 1/3
8601/8601 [==============================] - 15s 2ms/sample - loss: 0.1774 - acc: 0.9964 - val_loss: 0.0298 - val_acc: 1.0000
Epoch 2/3
8601/8601 [==============================] - 14s 2ms/sample - loss: 0.0244 - acc: 1.0000 - val_loss: 0.0199 - val_acc: 1.0000
Epoch 3/3
8601/8601 [==============================] - 14s 2ms/sample - loss: 0.0168 - acc: 1.0000 - val_loss: 0.0141 - val_acc: 1.0000
CPU times: user 1min 45s, sys: 43.3 s, total: 2min 28s
Wall time: 46 s


## PERFORMANCE ON THE TEST_SET


In [427]:
%%time
result = model.evaluate(x_test_pad, rep_train_x)

14448/14448 [==============================] - 13s 923us/sample - loss: 0.0071 - acc: 1.0000
CPU times: user 30 s, sys: 14.7 s, total: 44.8 s
Wall time: 13.3 s


In [428]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 100.00%
